<a href="https://colab.research.google.com/github/MSiswanto/NLP/blob/main/AIM0404D2203_Lab_1_Topic_Modelling_(Sklearn).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIM0404D2203 Lab 1 Topic Modelling (Sklearn)
---
<br>
<div style="font-size:20px;color:green;text-align:center"><strong>Dataset Dokumen Berita (Sklearn) </strong></div>
<br>
<img align=center src= "https://www.researchgate.net/publication/331669603/figure/fig1/AS:735488403070977@1552365508288/The-flow-of-LDA-Retrieved-from-https-toolboxkuriocoid-topic-modeling-696d7ba2592f.png">

<h1>Penjelasan Hands On</h1>

Pada pertemuan ini, kita akan belajar Topic Modeling menggunakan algoritme LDA (Latent Dirichlet Allocation). Kasus yang akan kita selesaikan yaitu terkait dokumen berita. Kita akan mengekstrak dokumen berita tersebut untuk mendapatkan topik-topik penting yang ada di dalamnya.

Proses menyelesaikan masalah di atas menggunakan pipeline pada Gambar di bawah ini:

1. Data collection 
2. Pre-processing
3. Feature Extraction
4. Menentukan Nilai K terbaik
5. Topic Modeling menggunakan LDA
6. Mencari kesimpulan terhadap topik yang terbentuk

<h1>Topic Modeling Pipeline</h1>
<img align=center src= "https://undangmi.com/wp-content/uploads/2022/09/LDA-pipeline.png">

# 01 Data Acquisition

In [ ]:
#Download dataset di bawah ini
#https://docs.google.com/spreadsheets/d/180ghjRmSRcR2iRVxBwZtVuhe7UumRcyD/edit?usp=sharing&ouid=117844591790417079121&rtpof=true&sd=true

In [ ]:
#Import library yang dibutuhkan
import re, string, unicodedata #modul regular expression
import nltk
from nltk import word_tokenize, sent_tokenize #Paket ini membagi teks input menjadi kata-kata.,
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#reading the dataset
data= pd.read_excel('/content/drive/MyDrive/Orbit Future Academy/dataset_documents_indonesia.xlsx')
data.head(5)

,Unnamed: 0,Article Title,Article Link,Article Content
0,1,\n 7 Hobi yang Bisa Datangkan C...,https://www.detik.com/edu/edutainment/d-588006...,Meski banyak orang menganggap hobi hanya seba...
1,2,"\n Perbanyak Ahli Gunung Api, I...",https://www.detik.com/edu/perguruan-tinggi/d-5...,Institut Teknologi Nasional (ITN) Yogyakarta m...
2,3,\n PTM Terbatas 2022 di Jakarta...,https://www.detik.com/edu/sekolah/d-5880075/pt...,Pemerintah Provinsi DKI Jakarta melalui Kepala...
3,4,\n Mengapa Perlu Menjaga Kelest...,https://www.detik.com/edu/detikpedia/d-5879161...,Tanaman bakau adalah salah satu ekosistem yang...
4,5,\n Cara Membuat Surat Lamaran K...,https://www.detik.com/edu/edutainment/d-587992...,Membuat Curriculum Vitae (CV) dan surat lamara...


# 02 Text Preprocessing

## Case Folding

In [ ]:
import re

# Buat fungsi untuk langkah case folding
def casefolding(text):
    text = text.lower()                               # Mengubah teks menjadi lower case
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Menghapus URL
    text = re.sub(r'[-+]?[0-9]+', '', text)           # Menghapus angka
    text = re.sub(r'[^\w\s]','', text)                # Menghapus karakter tanda baca
    text = text.strip()
    return text

In [ ]:
raw_sample = data['Article Content'].iloc[1]
case_folding = casefolding(raw_sample)

print('Raw data\t: ', raw_sample)
print('\n\nCase folding\t: ', case_folding)

Raw data	:  Institut Teknologi Nasional (ITN) Yogyakarta membuka magister Teknik Geologi dengan program unggulan kegunungapian. Program baru ini ditujukan meningkatkan jumlah ahli untuk melalukan penelitian terkait aktivitas gunung api. Rektor ITN Yogyakarta Dr. Ir. Ircham, M.T. mengatakan Indonesia merupakan negara kepulauan yang secara geografis terletak dalam kawasan kawasan ring of fire atau cincin api Pasifik. Menurut National Oceanic and Atmospheric Administration (NOAA), cincin api merupakan serangkaian 850-1.000 gunung berapi yang membentang hampir 40.250 kilometer di sekitar Samudra Pasifik. "Program ini akan memungkinkan semakin banyak penelitian yang akan dilakukan alumni ITN Yogyakarta memanfaatkan aktivitas gunung api bagi kemaslahatan kehidupan masyarakat," ujar Ircham dalam keterangan tertulisnya yang dikutip Sabtu, (1/1/2022). Ircham menyebut ITN Yogyakarta telah mendapat Surat Keputusan dari Kementerian Pendidikan, Kebudayaan, Riset, dan Teknologi (Kemendikbudristek) u

## Filtering (Stopword Removal)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stopwords_ind = stopwords.words('indonesian')

In [ ]:
len(stopwords_ind)

758

In [ ]:
# Lihat daftar stopword yang disediakan NLTK
stopwords_ind

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
# Buat fungsi untuk langkah stopword removal

more_stopword = ['national', 'oceanic']                    # Tambahkan kata dalam daftar stopword
stopwords_ind = stopwords_ind + more_stopword

def remove_stop_words(text):
    clean_words = []
    text = text.split()
    for word in text:
        if word not in stopwords_ind:
            clean_words.append(word)
    return " ".join(clean_words)

In [ ]:
remove_stop_words(case_folding)

'institut teknologi nasional itn yogyakarta membuka magister teknik geologi program unggulan kegunungapian program meningkatkan ahli melalukan penelitian terkait aktivitas gunung api rektor itn yogyakarta dr ir ircham mt indonesia negara kepulauan geografis terletak kawasan kawasan ring of fire cincin api pasifik and atmospheric administration noaa cincin api serangkaian gunung berapi membentang kilometer samudra pasifik program penelitian alumni itn yogyakarta memanfaatkan aktivitas gunung api kemaslahatan kehidupan masyarakat ircham keterangan tertulisnya dikutip sabtu ircham menyebut itn yogyakarta surat keputusan kementerian pendidikan kebudayaan riset teknologi kemendikbudristek membuka program magister program menarik mahasiswa mendalami ilmu geologi persen mahasiswa kampus persen mengikuti rumah target kuliah luring menunggu dikti komando ircham ahli gunung api surono gunung api aktif sekolah gunung api indonesia mempelajari vulkanologi menurutnya indonesia center of excellence 

In [ ]:
raw_sample = data['Article Content'].iloc[5]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)

print('Raw data\t: ', raw_sample)
print('\n\nCase folding\t: ', case_folding)
print('\n\nStopword removal\t: ', stopword_removal)

Raw data	:  Indonesia memiliki jam sekolah yang cukup panjang yaitu dimulai dari jam 7 pagi hingga jam 2 siang. Selain itu juga memiliki kegiatan yang cukup padat dan waktu istirahat yang pendek ditambah dengan adanya tambahan pekerjaan rumah untuk para siswa. Namun di beberapa negara menerapkan jam sekolah yang pendek. Di mana saja itu? Berikut penjelasannya yang dikutip oleh Insider: Menurut penelitian dari Pew Research Center, siswa sekolah dasar di Rusia bersekolah selama 470 jam di kelas selama satu tahun ajaran. Di Rusia waktu yang liburan juga cukup panjang, minimal pelajar Rusia libur selama empat bulan. Siswa di Rusia bersekolah lima hari seminggu dan biasanya mulai sekitar jam 8 pagi dan berakhir sekitar jam 1 atau 2 siang, Walaupun memiliki jam sekolah yang pendek, hal tersebut tidak berarti mereka belajar lebih sedikit. Sebanyak 99,7% orang berusia 15 tahun ke atas dapat membaca dan menulis di Rusia. Jam sekolah di Finlandia juga pendek. Siswa di Finlandia datang ke sekolah

In [ ]:
# Install library Sastrawi
!pip -q install sastrawi

     |████████████████████████████████| 209 kB 8.2 MB/s 


In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop_factory = StopWordRemoverFactory()
len(stop_factory.get_stop_words())

126

## Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Buat fungsi untuk langkah stemming bahasa Indonesia
def stemming(text):
    text = stemmer.stem(text)
    return text

In [ ]:
raw_sample       = data['Article Content'].iloc[5]
case_folding     = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)
text_stemming    = stemming(stopword_removal)

print('Raw data\t: ', raw_sample)
print('\n\nCase folding\t: ', case_folding)
print('\n\nStopword removal\t: ', stopword_removal)
print('\n\nStemming\t: ', text_stemming)

Raw data	:  Indonesia memiliki jam sekolah yang cukup panjang yaitu dimulai dari jam 7 pagi hingga jam 2 siang. Selain itu juga memiliki kegiatan yang cukup padat dan waktu istirahat yang pendek ditambah dengan adanya tambahan pekerjaan rumah untuk para siswa. Namun di beberapa negara menerapkan jam sekolah yang pendek. Di mana saja itu? Berikut penjelasannya yang dikutip oleh Insider: Menurut penelitian dari Pew Research Center, siswa sekolah dasar di Rusia bersekolah selama 470 jam di kelas selama satu tahun ajaran. Di Rusia waktu yang liburan juga cukup panjang, minimal pelajar Rusia libur selama empat bulan. Siswa di Rusia bersekolah lima hari seminggu dan biasanya mulai sekitar jam 8 pagi dan berakhir sekitar jam 1 atau 2 siang, Walaupun memiliki jam sekolah yang pendek, hal tersebut tidak berarti mereka belajar lebih sedikit. Sebanyak 99,7% orang berusia 15 tahun ke atas dapat membaca dan menulis di Rusia. Jam sekolah di Finlandia juga pendek. Siswa di Finlandia datang ke sekolah

## Text Preprocessing Pipeline

In [ ]:
# Buat fungsi untuk menggabungkan seluruh langkah text preprocessing
def text_preprocessing_process(text):
    text = casefolding(text)
    text = remove_stop_words(text)
    text = stemming(text)
    return text

In [ ]:
!pip install swifter #Install swifter. digunakan untuk meningkatkan kecepatan proses pandas
import swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 6.8 MB/s 
     |████████████████████████████████| 281 kB 47.0 MB/s 
     |████████████████████████████████| 1.6 MB 39.3 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16322 sha256=ffd085f5db3927f247f73fbc05b916bb97e25b57ee9dc22e32bb3c244b86c7a7
  Stored in directory: /root/.cache/pip/wheels/29/a7/0e/3a8f17ac69d759e1e93647114bc9bdc95957e5b0cbfd405205
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
%%time
data['clean_article'] = data['Article Content'].apply(text_preprocessing_process)

# Perhatikan waktu komputasi ketika proses text preprocessing

In [ ]:
data

In [ ]:
# save data preprocessing
# data.to_excel("dataset_preprocessing_news.xlsx")

# 03 Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect =TfidfVectorizer(max_features=1000)
vect_text=vect.fit_transform(data['clean_article'])

In [ ]:
#Menampilkan vocabulary dari TF-IDF
vect.vocabulary_

In [ ]:
#Lihat data tabular yang menggunakan metode TF-IDF
#Data ini siap untuk dimasukkan dalam proses machine learning

x1 = vect.transform(data['clean_article']).toarray()
data_tabular_tf_idf= pd.DataFrame(x1,columns=vect.get_feature_names())
data_tabular_tf_idf

# 04 LDA Model

In [ ]:
#Algoritme LDA (Latent Diriclet Allocation)
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,learning_method='online',random_state=0,max_iter=10) 

lda_top=lda_model.fit_transform(vect_text)

In [ ]:
#Melihat probabilitas dokumen terhadap topik
print("Document 1: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i+1,": ",topic*100,"%")

In [ ]:
data_cluster = []
for n in range(lda_top.shape[0]):
    topic_most_pr = lda_top[n].argmax()
    data_cluster.append(topic_most_pr)
    print("doc: {} topic: {}\n".format(n,topic_most_pr))
data ['cluster'] = data_cluster 

In [ ]:
vocab = vect.get_feature_names()
for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

In [ ]:
vocab = vect.get_feature_names()
for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i+1)+": ")
    for t in sorted_words:
        print(t,end=" ")
    print("\n")

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


for index, topic in enumerate(lda_model.components_):
    tmps = [vect.get_feature_names()[i] for i in topic.argsort()[-10:]]
    text = " ".join(str(topik) for topik in tmps)
    wordcloud = WordCloud(max_words=10,colormap='Set1', background_color="white").generate(text)
    plt.figure(figsize=(5,5))
    plt.imshow(wordcloud)
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.axis('off')
    plt.margins(x=0, y=0)
    plt.tight_layout()
    plt.show()
 

In [ ]:
data

# 05 Student Activity

Silahkan pahami dengan baik Topic Modeling pipeline di atas, kemudian kerjakan soal di bawah ini:

1. Terapkan Topic Modeling Pipeline di atas pada data yang berbeda dan tarik kesimpulan topik yang anda dapatkan. 
2. Data bisa menggunakan ini <a href="https://docs.google.com/spreadsheets/d/1OvKGTnyE3LnP-E6dtwOSLa1IK-ZdrR_i/edit?usp=sharing&ouid=117844591790417079121&rtpof=true&sd=true"> Klik</a>
3. Presentasikan hasil anda di kelas (Coach akan menunjuk beberapa perwakilan untuk presentasi). Persiapkan dengan baik yah.